<a href="https://colab.research.google.com/github/wiepat/Google-Colab/blob/main/Comparison_Rulebased_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NSL-KDD Datensatz einlesen und vorbereiten

Schritt 1: Google Colab vorbereiten, Datensatz laden und vorbereiten

In [ ]:
# -*- coding: utf-8 -*-
"""
NSL-KDD Datensatz einlesen, Regelbasiertes System und ML-Vergleich mit mehreren Algorithmen
"""
# 📌 1. Bibliotheken importieren
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix

# 📌 2. Spaltenbezeichnungen gemäß NSL-KDD Dokumentation
dataset_columns = [
    "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land",
    "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in", "num_compromised",
    "root_shell", "su_attempted", "num_root", "num_file_creations", "num_shells",
    "num_access_files", "num_outbound_cmds", "is_host_login", "is_guest_login",
    "count", "srv_count", "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
    "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate", "dst_host_count",
    "dst_host_srv_count", "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
    "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
    "dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate",
    "dst_host_srv_rerror_rate", "label", "difficulty_level"
]

# 📌 3. URLs für Trainings- und Testdatensätze
train_url = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt"
test_url = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest+.txt"

# 📌 4. Datensätze einlesen
df_train = pd.read_csv(train_url, names=dataset_columns, header=None)
df_test = pd.read_csv(test_url, names=dataset_columns, header=None)

# 📌 5. Entfernen der "difficulty_level" Spalte, da sie für das Training irrelevant ist
df_train.drop(columns=["difficulty_level"], inplace=True)
df_test.drop(columns=["difficulty_level"], inplace=True)

# 📌 6. Labels in binäre Werte umwandeln (0 = normal, 1 = Angriff)
df_train["label"] = df_train["label"].apply(lambda x: 0 if x == "normal" else 1)
df_test["label"] = df_test["label"].apply(lambda x: 0 if x == "normal" else 1)

# 📌 7. One-Hot-Encoding für kategoriale Features
categorical_columns = ["protocol_type", "service", "flag"]
df_train = pd.get_dummies(df_train, columns=categorical_columns, dtype=int)
df_test = pd.get_dummies(df_test, columns=categorical_columns, dtype=int)

# 📌 8. Sicherstellen, dass beide Datensätze dieselben Spalten haben
df_train, df_test = df_train.align(df_test, join='left', axis=1, fill_value=0)

# 📌 9. Spalte label aus Train- und Testdaten entfernen.
X_train = df_train.drop(columns=["label"])
y_train = df_train["label"]

X_test = df_test.drop(columns=["label"])
y_test = df_test["label"]

print("✅ NSL-KDD Datensätze erfolgreich eingelesen und verarbeitet!")

# 📌 9. Ausgabe der ersten 5 Zeilen
df_train.head(), df_test.head()


✅ NSL-KDD Datensätze erfolgreich eingelesen und verarbeitet!


(   duration  src_bytes  dst_bytes  land  wrong_fragment  urgent  hot  \
 0         0        491          0     0               0       0    0   
 1         0        146          0     0               0       0    0   
 2         0          0          0     0               0       0    0   
 3         0        232       8153     0               0       0    0   
 4         0        199        420     0               0       0    0   
 
    num_failed_logins  logged_in  num_compromised  ...  flag_REJ  flag_RSTO  \
 0                  0          0                0  ...         0          0   
 1                  0          0                0  ...         0          0   
 2                  0          0                0  ...         0          0   
 3                  0          1                0  ...         0          0   
 4                  0          1                0  ...         0          0   
 
    flag_RSTOS0  flag_RSTR  flag_S0  flag_S1  flag_S2  flag_S3  flag_SF  \
 0       

# Regelbasiertes System

Schritt 2: Regelbasiertes System

In [ ]:
# 📌 1. Regelbasierte Priorisierung von Sicherheitsvorfällen
def map_priority(row):
    """Simulierte regelbasierte Priorisierung"""
    if row["serror_rate"] > 0.5 or row["srv_serror_rate"] > 0.5:
        return 1  # Kritisch
    elif row["rerror_rate"] > 0.3 or row["srv_rerror_rate"] > 0.3:
        return 2  # Hoch
    elif row["count"] > 50:
        return 3  # Mittel
    else:
        return 4  # Gering

# 📌 2. Anwenden der Priorisierungsfunktion
df_test["rule_priority"] = df_test.apply(map_priority, axis=1)

# 📌 3. Regelbasierte Priorität in binäre Werte umwandeln (1-2 = Angriff, 3-4 = normal)
df_test["rule_priority_binary"] = np.where(df_test["rule_priority"] <= 2, 1, 0)

rule_precision = precision_score(y_test, df_test["rule_priority_binary"], zero_division=1)
rule_recall = recall_score(y_test, df_test["rule_priority_binary"])

# 📌 4. Zusätzliche Metriken berechnen
rule_f1 = f1_score(y_test, df_test["rule_priority_binary"])
rule_accuracy = accuracy_score(y_test, df_test["rule_priority_binary"])
rule_roc_auc = roc_auc_score(y_test, df_test["rule_priority_binary"])

# 📌 5. False Positive Rate (FPR) & False Negative Rate (FNR) berechnen
tn, fp, fn, tp = confusion_matrix(y_test, df_test["rule_priority_binary"]).ravel()
rule_fpr = fp / (fp + tn)
rule_fnr = fn / (fn + tp)

# 📌 6. Precision@100 berechnen (Wichtig für SOC-Priorisierung!)
def precision_at_k(y_true, y_pred, k=100):
    """Berechnet Precision@K für Priorisierungssysteme"""
    top_k = np.argsort(y_pred)[-k:]  # Nimmt die K höchsten Werte
    relevant = sum(y_true[top_k])  # Wie viele echte Incidents sind unter den Top-K?
    return relevant / k  # Precision@K

rule_p_at_100 = precision_at_k(y_test.to_numpy(), df_test["rule_priority_binary"].to_numpy(), k=100)

# 📌 7. Ergebnisse ausgeben
print(f"✅ Regelbasiert: Precision = {rule_precision:.2f}, Recall = {rule_recall:.2f}")
print(f"✅ Regelbasiert: F1-Score = {rule_f1:.2f}, Accuracy = {rule_accuracy:.2f}")
print(f"✅ Regelbasiert: ROC-AUC = {rule_roc_auc:.2f}")
print(f"✅ Regelbasiert: False Positive Rate (FPR) = {rule_fpr:.2f}")
print(f"✅ Regelbasiert: False Negative Rate (FNR) = {rule_fnr:.2f}")
print(f"✅ Regelbasiert: Precision@100 = {rule_p_at_100:.2f}")


✅ Regelbasiert: Precision = 0.99, Recall = 0.60
✅ Regelbasiert: F1-Score = 0.75, Accuracy = 0.77
✅ Regelbasiert: ROC-AUC = 0.80
✅ Regelbasiert: False Positive Rate (FPR) = 0.01
✅ Regelbasiert: False Negative Rate (FNR) = 0.40
✅ Regelbasiert: Precision@100 = 0.99


# Random Forest (ohne Tuning)

Random Forest trainieren und gegen Testdaten laufen lassen. Es wurde noch kein Feature Engineering, Feature Transformation, Feature Selection oder Hyperparameter-Tuning vorgenommen.

In [ ]:
# 📌 1. Machine Learning Modell (Random Forest)

# Die Spalten label, rule_priority and rule_priority_binary aus dem Test-
# datensatz entfernen.
X_test = df_test.drop(columns=["label", "rule_priority", "rule_priority_binary"])
y_test = df_test["label"]

# 📌 2. Modell trainieren
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 📌 3. Vorhersage auf Testdaten
rf_predictions = rf_model.predict(X_test)

# 📌 4. Performance-Metriken berechnen
rf_precision = precision_score(y_test, rf_predictions, zero_division=1)
rf_recall = recall_score(y_test, rf_predictions)

# 📌 5. Zusätzliche Metriken berechnen
rf_f1 = f1_score(y_test, rf_predictions)
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_roc_auc = roc_auc_score(y_test, rf_predictions)

# 📌 6. False Positive Rate (FPR) & False Negative Rate (FNR) berechnen
tn, fp, fn, tp = confusion_matrix(y_test, rf_predictions).ravel()
rf_fpr = fp / (fp + tn)
rf_fnr = fn / (fn + tp)

# 📌 7. Precision@100 berechnen (Wichtig für SOC-Priorisierung!)
def precision_at_k(y_true, y_pred, k=100):
    """Berechnet Precision@K für Priorisierungssysteme"""
    top_k = np.argsort(y_pred)[-k:]  # Nimmt die K höchsten Werte
    relevant = sum(y_true[top_k])  # Wie viele echte Incidents sind unter den Top-K?
    return relevant / k  # Precision@K

rf_p_at_100 = precision_at_k(y_test.to_numpy(), rf_predictions, k=100)

# 📌 8. Ergebnisse ausgeben
print(f"✅ Random Forest: Precision = {rf_precision:.2f}, Recall = {rf_recall:.2f}")
print(f"✅ Random Forest: F1-Score = {rf_f1:.2f}, Accuracy = {rf_accuracy:.2f}")
print(f"✅ Random Forest: ROC-AUC = {rf_roc_auc:.2f}")
print(f"✅ Random Forest: False Positive Rate (FPR) = {rf_fpr:.2f}")
print(f"✅ Random Forest: False Negative Rate (FNR) = {rf_fnr:.2f}")
print(f"✅ Random Forest: Precision@100 = {rf_p_at_100:.2f}")

✅ Random Forest: Precision = 0.97, Recall = 0.61
✅ Random Forest: F1-Score = 0.75, Accuracy = 0.77
✅ Random Forest: ROC-AUC = 0.79
✅ Random Forest: False Positive Rate (FPR) = 0.03
✅ Random Forest: False Negative Rate (FNR) = 0.39
✅ Random Forest: Precision@100 = 0.98


# Weitere ML-Modelle (ohne Tuning)

ML-Modell: XGBoost

In [ ]:
# 📌 1. ML-Modell: XGBoost
from xgboost import XGBClassifier

# 📌 2. XGBoost-Modell mit optimierten Einstellungen
xgb_model = XGBClassifier(n_estimators=200, max_depth=10, learning_rate=0.1,
                          scale_pos_weight=np.sum(y_train == 0) / np.sum(y_train == 1),
                          eval_metric="logloss", random_state=42)

print("🔍 Training XGBoost...")
xgb_model.fit(X_train, y_train)

# 📌 3. Vorhersage mit XGBoost
xgb_predictions = xgb_model.predict(X_test)

# 📌 4. Performance-Metriken für XGBoost berechnen
xgb_precision = precision_score(y_test, xgb_predictions, zero_division=1)
xgb_recall = recall_score(y_test, xgb_predictions)

# 📌 5. Zusätzliche Metriken berechnen
xgb_f1 = f1_score(y_test, xgb_predictions)
xgb_accuracy = accuracy_score(y_test, xgb_predictions)
xgb_roc_auc = roc_auc_score(y_test, xgb_predictions)

# 📌 6. False Positive Rate (FPR) & False Negative Rate (FNR) berechnen
tn, fp, fn, tp = confusion_matrix(y_test, xgb_predictions).ravel()
xgb_fpr = fp / (fp + tn)
xgb_fnr = fn / (fn + tp)

# 📌 7. Precision@100 berechnen (Wichtig für SOC-Priorisierung!)
def precision_at_k(y_true, y_pred, k=100):
    """Berechnet Precision@K für Priorisierungssysteme"""
    top_k = np.argsort(y_pred)[-k:]  # Nimmt die K höchsten Werte
    relevant = sum(y_true[top_k])  # Wie viele echte Incidents sind unter den Top-K?
    return relevant / k  # Precision@K

xgb_p_at_100 = precision_at_k(y_test.to_numpy(), xgb_predictions, k=100)


print(f"✅ XGBoost: Precision = {xgb_precision:.2f}, Recall = {xgb_recall:.2f}")


🔍 Training XGBoost...
✅ XGBoost: Precision = 0.97, Recall = 0.66


ML-Modell: LightGBM

In [ ]:
# 📌 1. ML-Modell: LightGBM

from lightgbm import LGBMClassifier


# 📌 2. LightGBM Modell
lgbm_model = LGBMClassifier(n_estimators=200, max_depth=10, learning_rate=0.1, random_state=42)
print("🔍 Training LightGBM...")
lgbm_model.fit(X_train, y_train)
lgbm_predictions = lgbm_model.predict(X_test)

# 📌 3. Performance-Metriken berechnen
lgbm_precision = precision_score(y_test, lgbm_predictions, zero_division=1)
lgbm_recall = recall_score(y_test, lgbm_predictions)

# 📌 4. Zusätzliche Metriken berechnen
lgbm_f1 = f1_score(y_test, lgbm_predictions)
lgbm_accuracy = accuracy_score(y_test, lgbm_predictions)
lgbm_roc_auc = roc_auc_score(y_test, lgbm_predictions)

# 📌 5. False Positive Rate (FPR) & False Negative Rate (FNR) berechnen
tn, fp, fn, tp = confusion_matrix(y_test, lgbm_predictions).ravel()
lgbm_fpr = fp / (fp + tn)
lgbm_fnr = fn / (fn + tp)

# 📌 6. Precision@100 berechnen (Wichtig für SOC-Priorisierung!)
def precision_at_k(y_true, y_pred, k=100):
    """Berechnet Precision@K für Priorisierungssysteme"""
    top_k = np.argsort(y_pred)[-k:]  # Nimmt die K höchsten Werte
    relevant = sum(y_true[top_k])  # Wie viele echte Incidents sind unter den Top-K?
    return relevant / k  # Precision@K

lgbm_p_at_100 = precision_at_k(y_test.to_numpy(), lgbm_predictions, k=100)


print(f"✅ LightGBM: Precision = {lgbm_precision:.2f}, Recall = {lgbm_recall:.2f}")


🔍 Training LightGBM...
[LightGBM] [Info] Number of positive: 58630, number of negative: 67343
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024365 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3296
[LightGBM] [Info] Number of data points in the train set: 125973, number of used features: 110
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.465417 -> initscore=-0.138552
[LightGBM] [Info] Start training from score -0.138552
✅ LightGBM: Precision = 0.97, Recall = 0.63


ML-Modell: CatBoost

In [3]:
# 📌 1. ML-Modell: CatBoost
!pip install catboost
from catboost import CatBoostClassifier

# 📌 2. CatBoost Modell
catboost_model = CatBoostClassifier(n_estimators=200, depth=10, learning_rate=0.1, verbose=0, random_state=42)
print("🔍 Training CatBoost...")
catboost_model.fit(X_train, y_train)
catboost_predictions = catboost_model.predict(X_test)

# 📌 3. Performance-Metriken berechnen
catboost_precision = precision_score(y_test, catboost_predictions, zero_division=1)
catboost_recall = recall_score(y_test, catboost_predictions)

# 📌 4. Zusätzliche Metriken berechnen
catboost_f1 = f1_score(y_test, catboost_predictions)
catboost_accuracy = accuracy_score(y_test, catboost_predictions)
catboost_roc_auc = roc_auc_score(y_test, catboost_predictions)

# 📌 5. False Positive Rate (FPR) & False Negative Rate (FNR) berechnen
tn, fp, fn, tp = confusion_matrix(y_test, catboost_predictions).ravel()
catboost_fpr = fp / (fp + tn)
catboost_fnr = fn / (fn + tp)

# 📌 6. Precision@100 berechnen (Wichtig für SOC-Priorisierung!)
def precision_at_k(y_true, y_pred, k=100):
    """Berechnet Precision@K für Priorisierungssysteme"""
    top_k = np.argsort(y_pred)[-k:]  # Nimmt die K höchsten Werte
    relevant = sum(y_true[top_k])  # Wie viele echte Incidents sind unter den Top-K?
    return relevant / k  # Precision@K

catboost_p_at_100 = precision_at_k(y_test.to_numpy(), catboost_predictions, k=100)


print(f"✅ CatBoost: Precision = {catboost_precision:.2f}, Recall = {catboost_recall:.2f}")

🔍 Training CatBoost...


NameError: name 'X_train' is not defined

ML-Modell: Isolation Forest

In [ ]:
# 📌 1. ML-Modell: Isolation Forest

from sklearn.ensemble import IsolationForest

# 📌 2. Isolation Forest Modell trainieren
iso_forest = IsolationForest(n_estimators=100, contamination=0.15, random_state=42)
iso_forest.fit(X_train)

# 📌 3. Anomalien im Testset erkennen (Anomalie = -1, Normal = 1)
iso_predictions = iso_forest.predict(X_test)
iso_predictions = np.where(iso_predictions == -1, 1, 0)  # Umwandeln in 1 (Angriff) oder 0 (Normal)

# 📌 4. Performance-Metriken für Isolation Forest berechnen
iso_precision = precision_score(y_test, iso_predictions, zero_division=1)
iso_recall = recall_score(y_test, iso_predictions)

# 📌 5. Zusätzliche Metriken berechnen
iso_f1 = f1_score(y_test, iso_predictions)
iso_accuracy = accuracy_score(y_test, iso_predictions)
iso_roc_auc = roc_auc_score(y_test, iso_predictions)

# 📌 6. False Positive Rate (FPR) & False Negative Rate (FNR) berechnen
tn, fp, fn, tp = confusion_matrix(y_test, iso_predictions).ravel()
iso_fpr = fp / (fp + tn)
iso_fnr = fn / (fn + tp)

# 📌 7. Precision@100 berechnen (Wichtig für SOC-Priorisierung!)
def precision_at_k(y_true, y_pred, k=100):
    """Berechnet Precision@K für Priorisierungssysteme"""
    top_k = np.argsort(y_pred)[-k:]  # Nimmt die K höchsten Werte
    relevant = sum(y_true[top_k])  # Wie viele echte Incidents sind unter den Top-K?
    return relevant / k  # Precision@K

iso_p_at_100 = precision_at_k(y_test.to_numpy(), iso_predictions, k=100)


print(f"✅ Isolation Forest: Precision = {iso_precision:.2f}, Recall = {iso_recall:.2f}")


✅ Isolation Forest: Precision = 0.86, Recall = 0.32


ML-Modell: One-Class SVM

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler

# 📌 1. Feature Selection: Wähle nur die wichtigsten Features
selected_features = X_train.columns[:20]  # Nutze die ersten 20 Features (kann angepasst werden)
X_train_reduced = X_train[selected_features]
X_test_reduced = X_test[selected_features]

# 📌 2. Standardisierung (Wichtig für SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reduced)
X_test_scaled = scaler.transform(X_test_reduced)

# 📌 3. Trainingsdaten reduzieren, um Geschwindigkeit zu erhöhen
X_train_sampled = X_train_scaled[:len(X_train_scaled) // 2]  # Nutze nur 50% der Daten

# 📌 4. One-Class SVM trainieren (mit optimierten Parametern)
ocsvm = OneClassSVM(kernel="rbf", gamma="scale", nu=0.1)
print("🔍 One-Class SVM Training gestartet...")
ocsvm.fit(X_train_sampled)  # Trainiere nur auf den reduzierten Daten
print("✅ One-Class SVM Training abgeschlossen!")

# 📌 5. Vorhersage auf Testdaten (-1 = Anomalie, 1 = Normal)
ocsvm_predictions = ocsvm.predict(X_test_scaled)
ocsvm_predictions = np.where(ocsvm_predictions == -1, 1, 0)  # Umwandeln in 1 (Angriff) oder 0 (Normal)

# 📌 6. Performance-Metriken für One-Class SVM berechnen
ocsvm_precision = precision_score(y_test, ocsvm_predictions, zero_division=1)
ocsvm_recall = recall_score(y_test, ocsvm_predictions)

# 📌 7. Zusätzliche Metriken berechnen
ocsvm_f1 = f1_score(y_test, ocsvm_predictions)
ocsvm_accuracy = accuracy_score(y_test, ocsvm_predictions)
ocsvm_roc_auc = roc_auc_score(y_test, ocsvm_predictions)

# 📌 8. False Positive Rate (FPR) & False Negative Rate (FNR) berechnen
tn, fp, fn, tp = confusion_matrix(y_test, ocsvm_predictions).ravel()
ocsvm_fpr = fp / (fp + tn)
ocsvm_fnr = fn / (fn + tp)

# 📌 9. Precision@100 berechnen (Wichtig für SOC-Priorisierung!)
def precision_at_k(y_true, y_pred, k=100):
    """Berechnet Precision@K für Priorisierungssysteme"""
    top_k = np.argsort(y_pred)[-k:]  # Nimmt die K höchsten Werte
    relevant = sum(y_true[top_k])  # Wie viele echte Incidents sind unter den Top-K?
    return relevant / k  # Precision@K

ocsvm_p_at_100 = precision_at_k(y_test.to_numpy(), ocsvm_predictions, k=100)

print(f"✅ One-Class SVM (Optimiert): Precision = {ocsvm_precision:.2f}, Recall = {ocsvm_recall:.2f}")


🔍 One-Class SVM Training gestartet...
✅ One-Class SVM Training abgeschlossen!
✅ One-Class SVM (Optimiert): Precision = 0.81, Recall = 0.32


ML-Modell: AutoEncoder

In [ ]:
# 📌 1.  ML-Modell: Autoencoder
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

# 📌 2. GPU-Check
print("🔍 Verfügbare GPUs:", tf.config.list_physical_devices('GPU'))

# 📌 3. Feature Scaling (Min-Max Normalisierung für 0-1 Werte)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train.iloc[:, :20])  # Nur Top-20 Features
X_test_scaled = scaler.transform(X_test.iloc[:, :20])

# 📌 4. Autoencoder-Modell definieren
input_dim = X_train_scaled.shape[1]
autoencoder = keras.Sequential([
    keras.layers.Dense(16, activation="relu", input_shape=(input_dim,)),  # Weniger Neuronen für Effizienz
    keras.layers.Dense(8, activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(input_dim, activation="sigmoid")  # Output entspricht Input-Dimension
])

autoencoder.compile(optimizer="adam", loss="mse")

# 📌 5. Training des Autoencoders
print("🔍 Starte Autoencoder-Training...")
autoencoder.fit(X_train_scaled, X_train_scaled, epochs=15, batch_size=128, shuffle=True, verbose=1)

# 📌 6. Fehler berechnen (je höher, desto wahrscheinlicher Anomalie)
reconstruction = autoencoder.predict(X_test_scaled)
mse = np.mean(np.power(X_test_scaled - reconstruction, 2), axis=1)

# 📌 7. Schwellenwert für Anomalien festlegen (90. Perzentil)
threshold = np.percentile(mse, 90)
autoencoder_predictions = (mse > threshold).astype(int)  # 1 = Angriff, 0 = Normal

# 📌 8. Performance-Metriken für Autoencoder berechnen
autoencoder_precision = precision_score(y_test, autoencoder_predictions, zero_division=1)
autoencoder_recall = recall_score(y_test, autoencoder_predictions)

# 📌 9. Zusätzliche Metriken berechnen
autoencoder_f1 = f1_score(y_test, autoencoder_predictions)
autoencoder_accuracy = accuracy_score(y_test, autoencoder_predictions)
autoencoder_roc_auc = roc_auc_score(y_test, autoencoder_predictions)

# 📌 10. False Positive Rate (FPR) & False Negative Rate (FNR) berechnen
tn, fp, fn, tp = confusion_matrix(y_test, autoencoder_predictions).ravel()
autoencoder_fpr = fp / (fp + tn)
autoencoder_fnr = fn / (fn + tp)

# 📌 11. Precision@100 berechnen (Wichtig für SOC-Priorisierung!)
def precision_at_k(y_true, y_pred, k=100):
    """Berechnet Precision@K für Priorisierungssysteme"""
    top_k = np.argsort(y_pred)[-k:]  # Nimmt die K höchsten Werte
    relevant = sum(y_true[top_k])  # Wie viele echte Incidents sind unter den Top-K?
    return relevant / k  # Precision@K

autoencoder_p_at_100 = precision_at_k(y_test.to_numpy(), autoencoder_predictions, k=100)


print(f"✅ Autoencoder: Precision = {autoencoder_precision:.2f}, Recall = {autoencoder_recall:.2f}")


🔍 Verfügbare GPUs: []
🔍 Starte Autoencoder-Training...
Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


985/985 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0772
Epoch 2/15
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0012
Epoch 3/15
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 9.7332e-04
Epoch 4/15
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 8.5384e-04
Epoch 5/15
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 8.4447e-04
Epoch 6/15
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 7.7869e-04
Epoch 7/15
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 7.4970e-04
Epoch 8/15
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 7.8314e-04
Epoch 9/15
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 7.5606e-04
Epoch 10/15
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 7.2862e-04
Epoch 11/15
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 7.7117e-04
Epoch 12/15
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 5.9371e-04
Epoch 13/15
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 5.6907e-04
Epoch 14/15
985/985 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 5.5168e-04
Epoch 15/15
985/985 

# Hybride Modelle

Regelbasiert & Random Forest

In [ ]:
# 📌 1. Hybrid-Modell: Kombination aus Regelbasiert & Random Forest
hybrid_rrf_predictions = df_test["rule_priority_binary"].copy()  # Starte mit regelbasierter Vorhersage

# 📌 2. Random Forest nur auf ungelabelte Fälle anwenden (wenn Regelbasiert = 0)
indices_ml_needed = df_test[df_test["rule_priority_binary"] == 0].index  # Wo Regelbasierte Methode unsicher ist

# 📌 3. RF-Vorhersagen für diese Fälle generieren
ml_predictions = rf_model.predict(X_test.loc[indices_ml_needed])

# 📌 4. Ersetze die Werte im Hybrid-Array nur dort, wo ML nötig ist
hybrid_rrf_predictions.loc[indices_ml_needed] = ml_predictions

# 📌 5. Performance-Metriken berechnen
hybrid_rrf_precision = precision_score(y_test, hybrid_rrf_predictions, zero_division=1)
hybrid_rrf_recall = recall_score(y_test, hybrid_rrf_predictions)
hybrid_rrf_f1 = f1_score(y_test, hybrid_rrf_predictions)
hybrid_rrf_accuracy = accuracy_score(y_test, hybrid_rrf_predictions)
hybrid_rrf_roc_auc = roc_auc_score(y_test, hybrid_rrf_predictions)

# 📌 6. False Positive Rate (FPR) & False Negative Rate (FNR) berechnen
tn, fp, fn, tp = confusion_matrix(y_test, hybrid_rrf_predictions).ravel()
hybrid_rrf_fpr = fp / (fp + tn)
hybrid_rrf_fnr = fn / (fn + tp)

# 📌 7. Precision@100 berechnen (Wichtig für SOC-Priorisierung!)
hybrid_rrf_p_at_100 = precision_at_k(y_test.to_numpy(), hybrid_rrf_predictions.to_numpy(), k=100)

# 📌 8. Ergebnisse ausgeben
print(f"✅ Hybrid (Regel + RF): Precision = {hybrid_rrf_precision:.2f}, Recall = {hybrid_rrf_recall:.2f}")
print(f"✅ Hybrid (Regel + RF): F1-Score = {hybrid_rrf_f1:.2f}, Accuracy = {hybrid_rrf_accuracy:.2f}")
print(f"✅ Hybrid (Regel + RF): ROC-AUC = {hybrid_rrf_roc_auc:.2f}")
print(f"✅ Hybrid (Regel + RF): False Positive Rate (FPR) = {hybrid_rrf_fpr:.2f}")
print(f"✅ Hybrid (Regel + RF): False Negative Rate (FNR) = {hybrid_rrf_fnr:.2f}")
print(f"✅ Hybrid (Regel + RF): Precision@100 = {hybrid_rrf_p_at_100:.2f}")



✅ Hybrid (Regel + RF): Precision = 0.97, Recall = 0.74
✅ Hybrid (Regel + RF): F1-Score = 0.84, Accuracy = 0.84
✅ Hybrid (Regel + RF): ROC-AUC = 0.85
✅ Hybrid (Regel + RF): False Positive Rate (FPR) = 0.03
✅ Hybrid (Regel + RF): False Negative Rate (FNR) = 0.26
✅ Hybrid (Regel + RF): Precision@100 = 0.95


XGBoost & Isolation Forest

In [ ]:
# 📌 1. Hybrides Modell: Kombination aus XGBoost & Isolation Forest
hybrid_xgbif_predictions = np.logical_or(xgb_predictions, iso_predictions).astype(int)

# 📌 2. Performance-Metriken berechnen
hybrid_xgbif_precision = precision_score(y_test, hybrid_xgbif_predictions, zero_division=1)
hybrid_xgbif_recall = recall_score(y_test, hybrid_xgbif_predictions)
hybrid_xgbif_f1 = f1_score(y_test, hybrid_xgbif_predictions)
hybrid_xgbif_accuracy = accuracy_score(y_test, hybrid_xgbif_predictions)
hybrid_xgbif_roc_auc = roc_auc_score(y_test, hybrid_xgbif_predictions)

# 📌 3. False Positive & False Negative Rate berechnen
tn, fp, fn, tp = confusion_matrix(y_test, hybrid_xgbif_predictions).ravel()
hybrid_xgbif_fpr = fp / (fp + tn)
hybrid_xgbif_fnr = fn / (fn + tp)

# 📌 4. Precision@100 berechnen
hybrid_xgbif_p_at_100 = precision_at_k(y_test.to_numpy(), hybrid_xgbif_predictions, k=100)


print(f"✅ Hybrid (XGBoost + Isolation Forest): Precision = {hybrid_xgbif_precision:.2f}, Recall = {hybrid_xgbif_recall:.2f}")


✅ Hybrid (XGBoost + Isolation Forest): Precision = 0.93, Recall = 0.73


LightGBM & One-Class SVM



In [ ]:
from lightgbm import LGBMClassifier
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler

# 📌 1. Feature Selection für schnelles Training
selected_features = X_train.columns[:20]  # Nutze nur die Top 20 Features
X_train_reduced = X_train[selected_features]
X_test_reduced = X_test[selected_features]

# 📌 2. Standardisierung (wichtig für One-Class SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reduced)
X_test_scaled = scaler.transform(X_test_reduced)

# 📌 3. One-Class SVM trainieren (nur auf normalem Traffic)
X_train_sampled = X_train_scaled[:len(X_train_scaled) // 2]  # Nutze nur 50 % für schnelleres Training
ocsvm = OneClassSVM(kernel="rbf", gamma="scale", nu=0.1)
print("🔍 Training One-Class SVM...")
ocsvm.fit(X_train_sampled)
print("✅ One-Class SVM Training abgeschlossen!")

# 📌 4. Vorhersage mit One-Class SVM (-1 = Anomalie, 1 = Normal)
ocsvm_predictions = ocsvm.predict(X_test_scaled)
ocsvm_predictions = np.where(ocsvm_predictions == -1, 1, 0)  # Anomalien als Angriff markieren

# 📌 5. LightGBM trainieren
lgbm_model = LGBMClassifier(n_estimators=200, max_depth=10, learning_rate=0.1, random_state=42)
print("🔍 Training LightGBM...")
lgbm_model.fit(X_train_reduced, y_train)
lgbm_predictions = lgbm_model.predict(X_test_reduced)
print("✅ LightGBM Training abgeschlossen!")

# 📌 6. Hybrid-Entscheidung: Falls One-Class SVM Angriff erkennt, setze 1, sonst nutze LightGBM
hybrid_lgbmsvm_predictions = ocsvm_predictions.copy()
indices_ml_needed = np.where(ocsvm_predictions == 0)[0]  # Wo OCSVM normal meldet → LGBM entscheidet

# 📌 7. LGBM-Vorhersagen nur für unklare Fälle übernehmen
hybrid_lgbmsvm_predictions[indices_ml_needed] = lgbm_predictions[indices_ml_needed]

# 📌 8. Performance-Metriken berechnen
hybrid_lgbmsvm_precision = precision_score(y_test, hybrid_lgbmsvm_predictions, zero_division=1)
hybrid_lgbmsvm_recall = recall_score(y_test, hybrid_lgbmsvm_predictions)
hybrid_lgbmsvm_f1 = f1_score(y_test, hybrid_lgbmsvm_predictions)
hybrid_lgbmsvm_accuracy = accuracy_score(y_test, hybrid_lgbmsvm_predictions)
hybrid_lgbmsvm_roc_auc = roc_auc_score(y_test, hybrid_lgbmsvm_predictions)

# 📌 9. False Positive & False Negative Rate berechnen
tn, fp, fn, tp = confusion_matrix(y_test, hybrid_lgbmsvm_predictions).ravel()
hybrid_lgbmsvm_fpr = fp / (fp + tn)
hybrid_lgbmsvm_fnr = fn / (fn + tp)

# 📌 10. Precision@100 berechnen
hybrid_lgbmsvm_p_at_100 = precision_at_k(y_test.to_numpy(), hybrid_lgbmsvm_predictions, k=100)

# 📌 11. Ergebnisse ausgeben
print(f"✅ Hybrid (LightGBM + OCSVM): Precision = {hybrid_lgbmsvm_precision:.2f}, Recall = {hybrid_lgbmsvm_recall:.2f}")
print(f"✅ Hybrid (LightGBM + OCSVM): F1-Score = {hybrid_lgbmsvm_f1:.2f}, Accuracy = {hybrid_lgbmsvm_accuracy:.2f}")
print(f"✅ Hybrid (LightGBM + OCSVM): ROC-AUC = {hybrid_lgbmsvm_roc_auc:.2f}")
print(f"✅ Hybrid (LightGBM + OCSVM): False Positive Rate (FPR) = {hybrid_lgbmsvm_fpr:.2f}")
print(f"✅ Hybrid (LightGBM + OCSVM): False Negative Rate (FNR) = {hybrid_lgbmsvm_fnr:.2f}")
print(f"✅ Hybrid (LightGBM + OCSVM): Precision@100 = {hybrid_lgbmsvm_p_at_100:.2f}")


🔍 Training One-Class SVM...
✅ One-Class SVM Training abgeschlossen!
🔍 Training LightGBM...
[LightGBM] [Info] Number of positive: 58630, number of negative: 67343
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007056 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1164
[LightGBM] [Info] Number of data points in the train set: 125973, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.465417 -> initscore=-0.138552
[LightGBM] [Info] Start training from score -0.138552
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
✅ LightGBM Training abgeschlossen!
✅ Hybrid (LightGBM + OCSVM): Precision = 0.90, Recall = 0.80
✅ Hybrid (LightGBM + OCSVM): F1-Score = 0.85, Accuracy = 0.83
✅ Hybrid (LightGBM + OCSVM): ROC-AUC = 0.84
✅ Hybrid (LightGBM + OCSVM): False Positive Rate (FPR) = 0.12
✅ Hybrid (L

# Ergebnisse

Modelle ohne Tuning vergleichen

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 📌 1. Ergebnisse als DataFrame speichern
results = pd.DataFrame({
    "Modell": [
        "Regelbasiert", "Random Forest", "XGBoost", "LightGBM", "CatBoost",
        "Isolation Forest", "One-Class SVM", "Autoencoder",
        "Hybrid (Regel + RF)", "Hybrid (XGBoost + Isolation Forest)", "Hybrid (LightGBM + OCSVM)"
    ],
    "Precision": [
        rule_precision, rf_precision, xgb_precision, lgbm_precision, catboost_precision,
        iso_precision, ocsvm_precision, autoencoder_precision,
        hybrid_rrf_precision, hybrid_xgbif_precision, hybrid_lgbmsvm_precision
    ],
    "Recall": [
        rule_recall, rf_recall, xgb_recall, lgbm_recall, catboost_recall,
        iso_recall, ocsvm_recall, autoencoder_recall,
        hybrid_rrf_recall, hybrid_xgbif_recall, hybrid_lgbmsvm_recall
    ],
    "F1-Score": [
        rule_f1, rf_f1, xgb_f1, lgbm_f1, catboost_f1,
        iso_f1, ocsvm_f1, autoencoder_f1,
        hybrid_rrf_f1, hybrid_xgbif_f1, hybrid_lgbmsvm_f1
    ],
    "Accuracy": [
        rule_accuracy, rf_accuracy, xgb_accuracy, lgbm_accuracy, catboost_accuracy,
        iso_accuracy, ocsvm_accuracy, autoencoder_accuracy,
        hybrid_rrf_accuracy, hybrid_xgbif_accuracy, hybrid_lgbmsvm_accuracy
    ],
    "ROC-AUC": [
        rule_roc_auc, rf_roc_auc, xgb_roc_auc, lgbm_roc_auc, catboost_roc_auc,
        None, None, None,  # Isolation Forest & Autoencoder haben kein klassisches ROC-AUC
        hybrid_rrf_roc_auc, hybrid_xgbif_roc_auc, hybrid_lgbmsvm_roc_auc
    ],
    "False Positive Rate (FPR)": [
        rule_fpr, rf_fpr, xgb_fpr, lgbm_fpr, catboost_fpr,
        iso_fpr, ocsvm_fpr, autoencoder_fpr,
        hybrid_rrf_fpr, hybrid_xgbif_fpr, hybrid_lgbmsvm_fpr
    ],
    "False Negative Rate (FNR)": [
        rule_fnr, rf_fnr, xgb_fnr, lgbm_fnr, catboost_fnr,
        iso_fnr, ocsvm_fnr, autoencoder_fnr,
        hybrid_rrf_fnr, hybrid_xgbif_fnr, hybrid_lgbmsvm_fnr
    ],
    "Precision@100": [
        rule_p_at_100, rf_p_at_100, xgb_p_at_100, lgbm_p_at_100, catboost_p_at_100,
        iso_p_at_100, ocsvm_p_at_100, autoencoder_p_at_100,
        hybrid_rrf_p_at_100, hybrid_xgbif_p_at_100, hybrid_lgbmsvm_p_at_100
    ]
})

# 📌 2. Tabelle in Google Colab anzeigen
from IPython.display import display
display(results)


,Modell,Precision,Recall,F1-Score,Accuracy,ROC-AUC,False Positive Rate (FPR),False Negative Rate (FNR),Precision@100
0,Regelbasiert,0.986499,0.603522,0.748888,0.769606,0.796303,0.010915,0.396478,0.99
1,Random Forest,0.967030,0.607964,0.746567,0.765037,0.790286,0.027392,0.392036,0.98
2,XGBoost,0.968892,0.662589,0.786987,0.795822,0.817238,0.028112,0.337411,0.95
3,LightGBM,0.967197,0.629549,0.762673,0.776969,0.800667,0.028215,0.370451,0.94
4,CatBoost,0.968875,0.662199,0.786706,0.795600,0.817043,0.028112,0.337801,0.95
5,Isolation Forest,0.859679,0.316995,0.463194,0.581751,NaN,0.068376,0.683005,0.92
6,One-Class SVM,0.805408,0.318008,0.455978,0.568045,NaN,0.101534,0.681992,0.85
7,Autoencoder,0.654545,0.115016,0.195652,0.461675,NaN,0.080218,0.884984,0.66
8,Hybrid (Regel + RF),0.967900,0.737785,0.837320,0.836808,0.852725,0.032334,0.262215,0.95
9,Hybrid (XGBoost + Isolation Forest),0.927244,0.734902,0.819944,0.816270,0.829350,0.076202,0.265098,0.91
